# Offline Balance Load Team Assigment  
## Implementation of Randomized LP-relaxation 
## proposed by Aris

__________________

### I. Helper Functions

In [201]:
import numpy as np 
from pulp import *
import random as rd

# cross products
def cross(a,b):
    p = 0
    for (x,y) in zip(a,b):
        p+=(x*y)
    return p

# show outputs
def output(arr):
    new = np.zeros((len(arr),len(arr[0])))
    for i in range(len(new)):
        for j in range(len(new[0])):
            new[i][j] = value(arr[i][j])
    return new

# print constraints
def printCons(prob):
    for a in prob.constraints.values():
        print(a)

### II. Lp-solving stage

In [232]:
# input 
# J = np.array([[1,0,0],[1,1,1],[0,1,0]]) # 3 tasks
# P = np.array([[1,1,0],[0,0,1],[0,1,0]]) # 2 people 

J = np.array([[1,1,1]]) # 3 tasks
P = np.array([[1,1,0],[1,0,1],[0,1,1]]) # 2 people 

X = [[0 for x in range(len(J))] for y in range(len(P))]  # assignment matrix

# declare your variables
L = LpVariable("L", 0, 1000)
for i in range(len(X)):
    for j in range(len(X[1])):
        X[i][j] = LpVariable("x"+str(i)+str(j), 0, 1) # 0=<x<=1

In [233]:
# defines the problem
prob = LpProblem("problem", LpMinimize)

# defines the objective function to minimize
prob += L

In [234]:
# defines the constraints

for i in range(len(X)):  # all people's loads subject to a uppper bound 
    prob += sum(X[i])<=L
    
for i in range(len(J)): # all skills in all tasks must be covered 
    for j in range(len(J[0])):
        prob += cross([a[i] for a in X],[a[j] for a in P]) >= J[i][j]
printCons(prob)

-L + x00 <= 0
-L + x10 <= 0
-L + x20 <= 0
x00 + x10 >= 1
x00 + x20 >= 1
x10 + x20 >= 1


______

#### Additional constraint!!!

In [235]:
# add additional constrain to lp problem: 
# person p can be only assigned to team j
def prerequsite(p,j,prob,X): 
    prob += X[p][j] == 1
    prob += sum(X[p]) == 1

# test 
prerequsite(0,0,prob,X)
printCons(prob)

-L + x00 <= 0
-L + x10 <= 0
-L + x20 <= 0
x00 + x10 >= 1
x00 + x20 >= 1
x10 + x20 >= 1
x00 = 1
x00 = 1


______________

In [236]:
 # solve the problem
status = prob.solve(GLPK(msg=0))
LpStatus[status]

'Optimal'

In [237]:
# print output 
print("L:",value(L))
print("X:\n",output(X))

L: 1.0
X:
 [[ 1.]
 [ 1.]
 [ 0.]]


### III. Randomize stage

In [238]:
X = output(X)
R = 10
for i in range(len(X)):
    for j in range(len(X[0])):
        for k in range(R):
            if rd.uniform(0,1)<X[i][j]: # if at least one time it's chosen
                X[i][j] = 1 
                break
            if k == R-1: X[i][j] = 0 # if not chosen in any round

In [239]:
print("L:", max([sum(a) for a in X]))
print("X:", X)

L: 1.0
X: [[ 1.]
 [ 1.]
 [ 0.]]
